In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pickle
import os
import seaborn as sns
from tqdm.auto import tqdm

## 集大成！

In [9]:
# Define the folder path and file naming convention
folder_path = '/scratch/work/lyub2/2.df_tc_09'
file_prefix = 'df_tc_'
file_suffix = '.csv'

# Columns to check for missing values
columns_to_check = ['leg_order', 'id', 'activity', 'km', 'time_start_distance', 'time_end_distance']
cols = ['leg_order', 'id', 'activity', 'km', 'time_start_distance', 'time_end_distance', 'locationAvailable']

# Initialize accumulators for total counts
total_location_counts = pd.Series([0, 0, 0, 0], index=['Always', 'Denied', 'Unknown', 'WhenInUse'])
total_missing_counts = pd.DataFrame(0, index=columns_to_check, columns=['Always_missing_counts', 'Denied_missing_counts', 'Unknown_missing_counts', 'WhenInUse_missing_counts'])

# Loop through each CSV file
for i in tqdm(range(1, 31), desc="Processing files"):
    file_name = f'{file_prefix}{i:02d}{file_suffix}'
    file_path = os.path.join(folder_path, file_name)
    
    # Check if the file exists to avoid errors
    if not os.path.isfile(file_path):
        print(f"File not found: {file_path}")
        continue
    
    # Read the CSV file
    df_tc = pd.read_csv(file_path, usecols=cols)
    
    # Get the locationAvailable counts for this file
    location_counts = df_tc['locationAvailable'].value_counts()
    location_counts = location_counts.reindex(['Always', 'Denied', 'Unknown', 'WhenInUse'], fill_value=0)
    
    # Update the total counts for locationAvailable
    total_location_counts += location_counts

    # Group by 'locationAvailable' and count missing values for each column
    missing_values_location = df_tc.groupby('locationAvailable')[columns_to_check].apply(lambda x: x.isna().sum())

    # Update the total missing counts
    total_missing_counts['Always_missing_counts'] += missing_values_location.loc['Always', columns_to_check].values
    total_missing_counts['Denied_missing_counts'] += missing_values_location.loc['Denied', columns_to_check].values
    total_missing_counts['Unknown_missing_counts'] += missing_values_location.loc['Unknown', columns_to_check].values
    total_missing_counts['WhenInUse_missing_counts'] += missing_values_location.loc['WhenInUse', columns_to_check].values

# Calculate the overall missing percentages after accumulating all counts
total_missing_percentages = pd.DataFrame(0, index=columns_to_check, columns=['Always_missing_%', 'Denied_missing_%', 'Unknown_missing_%', 'WhenInUse_missing_%'])
total_missing_percentages['Always_missing_%'] = round((total_missing_counts['Always_missing_counts'] / total_location_counts['Always']) * 100, 2)
total_missing_percentages['Denied_missing_%'] = round((total_missing_counts['Denied_missing_counts'] / total_location_counts['Denied']) * 100, 2)
total_missing_percentages['Unknown_missing_%'] = round((total_missing_counts['Unknown_missing_counts'] / total_location_counts['Unknown']) * 100, 2)
total_missing_percentages['WhenInUse_missing_%'] = round((total_missing_counts['WhenInUse_missing_counts'] / total_location_counts['WhenInUse']) * 100, 2)

# Create the final DataFrame combining counts and percentages
df_a4 = pd.concat([total_missing_counts, total_missing_percentages], axis=1)

print('DONE!!!')

Processing files:   0%|          | 0/30 [00:00<?, ?it/s]

DONE!!!


In [10]:
df_a4

,Always_missing_counts,Denied_missing_counts,Unknown_missing_counts,WhenInUse_missing_counts,Always_missing_%,Denied_missing_%,Unknown_missing_%,WhenInUse_missing_%
leg_order,13234228,578,201,15310020,9.57,9.63,9.12,9.94
id,13244026,578,201,15318304,9.58,9.63,9.12,9.94
activity,13244026,578,201,15318304,9.58,9.63,9.12,9.94
km,13244026,578,201,15318304,9.58,9.63,9.12,9.94
time_start_distance,13244026,578,201,15318304,9.58,9.63,9.12,9.94
time_end_distance,13244026,578,201,15318304,9.58,9.63,9.12,9.94


In [6]:
df_a4.to_csv('/scratch/work/lyub2/Problem_output_September/P1A4/df_a4.csv')

In [7]:
total_location_counts

Always       138289873
Denied            6004
Unknown           2203
WhenInUse    154053451
dtype: int64

### Plot